## Convert Aaron's txt timestamps to .mat files and create Epochs object
### Input: Labeled timestamps in .txt files
###  Output: *-epo.fif, *-epo_times.mat, and *-stim_labels.mat

In [1]:
sub_num = "040"

## Load in data

Define some functions for finding relevant events + event name dictionary

In [2]:
import sys
import os
import numpy as np
import mne
import matplotlib.pyplot as plt

from mne.preprocessing import ICA
from pyprep.find_noisy_channels import NoisyChannels

from autoreject import AutoReject
from autoreject import get_rejection_threshold

from IPython import display

from scipy.io import savemat

## define label dictionary for epoch annotations
## different keys account for different naming conventions
## NOTE: the Trigger#X naming convention does not specify between hand and back stimulus
custom_mapping = {'eyes closed': 1, 'Trigger#1': 1, 'EYES CLOSED': 1, # eyes closed
                  'eyes open': 2, 'eyes opened': 2, 'Trigger#2': 2, 'EYES OPEN': 2, 'eyes openned':2, # eyes open
                  'pinprick hand': 3, 'hand pinprick':3, 'Yes Pain Hand': 3, 'Trigger#3': 3, 'HAND PINPRICK': 3, 'hand 32 gauge pinprick': 3, 'Yes Hand Pain': 3,
                  # highest intensity pain stimulus (first two HAND)
                  'Med Pain Hand': 4 , 'Med Hand Pain': 4 , # intermediate intensity pain stimulus (HAND)
                  'No Pain Hand': 5, 'hand plastic':5, 'plastic hand': 5, 'Trigger#4': 5,  'HAND PLASTIC': 5, 'hand plastic filament':5, 'No Hand Pain': 5, 
                  # sensory stimulus, no pain (first two HAND)
                  'pinprick back': 6, 'back pinprick':6, 'Yes Pain Back': 6, 'BACK  PINPRICK': 6, 'Trigger#5': 6, 'back 32 gauge pinprick':6, 'Yes Back Pain': 6,
                  # highest intensity pain stimulus (BACK)
                  'Med Pain Back': 7, 'Med Back Pain': 7, # intermediate intensity pain stimulus (BACK)
                  'plastic back': 8, 'back plastic':8, 'No Pain Back': 8, 'BACK PLASTIC': 8, 'Trigger#6': 8, 'back plastic filament': 8 , 'No Back Pain': 8,
                  # sensory stimulus, no pain (BACK)
                  'stop': 9, 'Stop': 9, 'STOP': 9, # stop
                  '1000001': 10,  '100160': 10, '100480': 10, # lesser weight pen tip down
                  '1000010': 11, # lesser weight pen tip up
                  '1100001': 12, '100320': 12, '100480': 12, # greater weight pen tip down
                  '1100010': 13, # greater weight pen tip up
                  }

Import continuous FIF

In [3]:
# subject ID
data_dir = "../../../Matlab Code/Processed Data/"
# data_dir = "../../../Matlab Code/Processed Data - Needing Marker Repair/"

# sub_num = sys.argv[1]

sub_id=''
for file in os.listdir(data_dir):
    if file.startswith(sub_num) and file.endswith('raw.fif'):
        sub_id = file

print(f"{sub_id}\nreading preprocessed-raw file...")
raw = mne.io.read_raw_fif(data_dir+sub_id, preload=True)

res_freq = raw.info['sfreq'] # save resampled sampling frequency

(events_from_annot, event_dict) = mne.events_from_annotations(raw, event_id=custom_mapping)

# get key and val lists from event_dict
key_list = list(event_dict.keys())
val_list = list(event_dict.values())

raw

040_preprocessed_avg-raw.fif
reading preprocessed-raw file...
Opening raw data file ../../../Matlab Code/Processed Data/040_preprocessed_avg-raw.fif...
    Range : 0 ... 535839 =      0.000 ...  1339.598 secs
Ready.
Reading 0 ... 535839  =      0.000 ...  1339.598 secs...
Used Annotations descriptions: ['1000001', '1000010', '1100001', '1100010', 'Med Pain Back', 'Med Pain Hand', 'No Pain Back', 'No Pain Hand', 'Stop', 'Yes Pain Back', 'Yes Pain Hand', 'eyes closed', 'eyes opened']


<Raw | 040_preprocessed_avg-raw.fif, 64 x 535840 (1339.6 s), ~261.7 MB, data loaded>

Get original sampling frequency (orig_freq)

In [4]:
orig_data_dir = "../../../EEG DATA/"
sub_id=''; acq_id=''
for folder in os.listdir(orig_data_dir):
    if folder.startswith(sub_num):
        sub_id = folder;
for subfile in os.listdir(os.path.join(orig_data_dir,sub_id)):
    if subfile.endswith(".edf"):
        acq_id = subfile
eeg_data_raw_file = os.path.join(orig_data_dir,sub_id,acq_id)
orig_raw = mne.io.read_raw_edf(eeg_data_raw_file, preload=False)
display.clear_output(wait=True)

orig_freq = orig_raw.info['sfreq']
print('orig_freq: ', orig_freq)

orig_freq:  1000.0


Validate channel names

In [5]:
print(raw.ch_names)

['Fp1', 'Fpz', 'Fp2', 'AF3', 'AF4', 'F11', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'F12', 'FT11', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT12', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'M1', 'M2', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Cb1', 'Cb2']


Check event labels and see whether either key presses or pinprick markers are missing

In [6]:
desc_full = raw.annotations.description
desc_full = desc_full.tolist()
print(len(desc_full))
print(raw.annotations.description[200:250])

540
['No Pain Hand' '1100001' '1100010' '1100001' '1100010' '1100001'
 '1100010' '1100001' '1100010' '1100001' '1100010' '1100001' '1100010'
 'No Pain Hand' '1100001' '1100010' '1100001' '1100010' 'Yes Pain Hand'
 '1000001' '1000010' '1000001' '1000010' 'No Pain Hand' '1100001'
 '1100010' '1100001' '1100010' 'Yes Pain Hand' '1000001' '1000010'
 '1000001' '1000010' '1000001' '1000010' 'Yes Pain Hand' '1000001'
 '1000010' 'Yes Pain Back' '1000001' '1000010' '1000001' '1000010'
 '1000001' '1000010' '1000001' '1000010' 'Yes Pain Back' '1000001'
 '1000010']


In [7]:
# # create events to epochize data
# print(f"{sub_id}\ncreating events and epochizing data...")
# (events_from_annot, event_dict) = mne.events_from_annotations(raw, event_id=custom_mapping)

# # get key and val lists from event_dict
# key_list = list(event_dict.keys())
# val_list = list(event_dict.values())

# epochs = mne.Epochs(raw, events_from_annot, event_dict, tmin=-0.2, tmax=0.8, proj=True, preload=True,
#                     event_repeated='drop')#, baseline=(None,0))
# display.clear_output(wait=True)

# epochs

# # del raw # clear memory

## Load in .txt file

In [9]:
# IMPORT DICT WITH EVENT TIMES CREATED BY AARON

import json
  
txt_dir = "../../Re__EEG_BCI_Project/"

# reading the data from the file
with open(txt_dir+sub_num[:3]+".txt") as f:
    data = f.read()
    data = data.replace("'", '"')

print("Data type before reconstruction : ", type(data))
      
# reconstructing the data as a dictionary
js = json.loads(data)
  
print("Data type after reconstruction : ", type(js))
print(js)

Data type before reconstruction :  <class 'str'>
Data type after reconstruction :  <class 'dict'>
{'No Pain Hand': [793514, 810065, 815845, 836574, 842263, 970714, 988803, 1006030, 1010686, 1025950], 'Med Pain Hand': [782882, 800810, 822903, 849715, 855031, 1172939, 1189615, 1207044, 1211915, 1227031], 'Yes Pain Hand': [980273, 996952, 1020144, 1034250, 1038958, 1157433, 1163225, 1194935, 1221280, 1234585], 'No Pain Back': [869068, 877200, 903315, 928169, 946597, 1066960, 1075737, 1093066, 1111296, 1128097], 'Med Pain Back': [885202, 894064, 911719, 917917, 935719, 1252519, 1277194, 1306179, 1320372, 1329104], 'Yes Pain Back': [1050112, 1058443, 1085396, 1104517, 1120961, 1259576, 1266695, 1283634, 1290374, 1313201]}


In [10]:
# Create lists for events and labels from Aaron's txt file
txt_samps_list = [item for sublist in list(js.values()) for item in sublist]
print(txt_samps_list)
print(len(txt_samps_list))

[793514, 810065, 815845, 836574, 842263, 970714, 988803, 1006030, 1010686, 1025950, 782882, 800810, 822903, 849715, 855031, 1172939, 1189615, 1207044, 1211915, 1227031, 980273, 996952, 1020144, 1034250, 1038958, 1157433, 1163225, 1194935, 1221280, 1234585, 869068, 877200, 903315, 928169, 946597, 1066960, 1075737, 1093066, 1111296, 1128097, 885202, 894064, 911719, 917917, 935719, 1252519, 1277194, 1306179, 1320372, 1329104, 1050112, 1058443, 1085396, 1104517, 1120961, 1259576, 1266695, 1283634, 1290374, 1313201]
60


In [11]:
# Create corresponding list of labels
txt_labels_list = []
for key in js:
    txt_labels_list.append([key]*(len(js[key])))
    
txt_labels_list = [item for sublist in txt_labels_list for item in sublist]

print(txt_labels_list)
print(len(txt_labels_list))

['No Pain Hand', 'No Pain Hand', 'No Pain Hand', 'No Pain Hand', 'No Pain Hand', 'No Pain Hand', 'No Pain Hand', 'No Pain Hand', 'No Pain Hand', 'No Pain Hand', 'Med Pain Hand', 'Med Pain Hand', 'Med Pain Hand', 'Med Pain Hand', 'Med Pain Hand', 'Med Pain Hand', 'Med Pain Hand', 'Med Pain Hand', 'Med Pain Hand', 'Med Pain Hand', 'Yes Pain Hand', 'Yes Pain Hand', 'Yes Pain Hand', 'Yes Pain Hand', 'Yes Pain Hand', 'Yes Pain Hand', 'Yes Pain Hand', 'Yes Pain Hand', 'Yes Pain Hand', 'Yes Pain Hand', 'No Pain Back', 'No Pain Back', 'No Pain Back', 'No Pain Back', 'No Pain Back', 'No Pain Back', 'No Pain Back', 'No Pain Back', 'No Pain Back', 'No Pain Back', 'Med Pain Back', 'Med Pain Back', 'Med Pain Back', 'Med Pain Back', 'Med Pain Back', 'Med Pain Back', 'Med Pain Back', 'Med Pain Back', 'Med Pain Back', 'Med Pain Back', 'Yes Pain Back', 'Yes Pain Back', 'Yes Pain Back', 'Yes Pain Back', 'Yes Pain Back', 'Yes Pain Back', 'Yes Pain Back', 'Yes Pain Back', 'Yes Pain Back', 'Yes Pain Back']

## Convert txt_times_list and txt_labels_list to appropriate format for Epochs object

In [12]:
# Sort lists in chronological order and permute event and label lists accordingly

txt_times_arr = np.array(txt_samps_list)
sort_index = np.argsort(txt_samps_list)
print(sort_index)

# Permute times and labels lists:
txt_samps_list = [txt_samps_list[i] for i in sort_index]
txt_labels_list = [txt_labels_list[i] for i in sort_index]

print(txt_samps_list[:5])
print(txt_labels_list[:5])

[10  0 11  1  2 12  3  4 13 14 30 31 40 41 32 42 43 33 44 34  5 20  6 21
  7  8 22  9 23 24 50 51 35 36 52 37 53 38 54 39 25 26 15 16 27 17 18 28
 19 29 45 55 56 46 57 58 47 59 48 49]
[782882, 793514, 800810, 810065, 815845]
['Med Pain Hand', 'No Pain Hand', 'Med Pain Hand', 'No Pain Hand', 'No Pain Hand']


In [13]:
# define labels in separate lists
custom_mapping.keys()
# hand
yes_hand_pain_list = list(custom_mapping.keys())[8:15]
print(yes_hand_pain_list)

med_hand_pain_list = list(custom_mapping.keys())[15:17]
print(med_hand_pain_list)

no_hand_pain_list = list(custom_mapping.keys())[17:24]
print(no_hand_pain_list)

# back
yes_back_pain_list = list(custom_mapping.keys())[24:31]
print(yes_back_pain_list)

med_back_pain_list = list(custom_mapping.keys())[31:33]
print(med_back_pain_list)

no_back_pain_list = list(custom_mapping.keys())[33:40]
print(no_back_pain_list)

['pinprick hand', 'hand pinprick', 'Yes Pain Hand', 'Trigger#3', 'HAND PINPRICK', 'hand 32 gauge pinprick', 'Yes Hand Pain']
['Med Pain Hand', 'Med Hand Pain']
['No Pain Hand', 'hand plastic', 'plastic hand', 'Trigger#4', 'HAND PLASTIC', 'hand plastic filament', 'No Hand Pain']
['pinprick back', 'back pinprick', 'Yes Pain Back', 'BACK  PINPRICK', 'Trigger#5', 'back 32 gauge pinprick', 'Yes Back Pain']
['Med Pain Back', 'Med Back Pain']
['plastic back', 'back plastic', 'No Pain Back', 'BACK PLASTIC', 'Trigger#6', 'back plastic filament', 'No Back Pain']


Change string labels to keys

In [ ]:
# change labels to keys
def labels_to_keys(txt_labels_list, val_list):
    stim_labels = [0]*len(txt_labels_list)
    if 10 in val_list or 11 in val_list or 12 in val_list or 13 in val_list:
        for i in range(0,len(stim_labels)):
            if txt_labels_list[i] in yes_hand_pain_list:
                    stim_labels[i] = 3
            elif txt_labels_list[i] in med_hand_pain_list:
                    stim_labels[i] = 4
            elif txt_labels_list[i] in no_hand_pain_list:
                    stim_labels[i] = 5
            elif txt_labels_list[i] in yes_back_pain_list:
                    stim_labels[i] = 6
            elif txt_labels_list[i] in med_back_pain_list:
                    stim_labels[i] = 7
            elif txt_labels_list[i] in no_back_pain_list:
                    stim_labels[i] = 8
                
    if 10 not in val_list or 11 not in val_list or 12 not in val_list or 13 not in val_list:
        for i in range(0,len(stim_labels)):
            if txt_labels_list[i] in yes_hand_pain_list:
                stim_labels[i] = 3
            elif txt_labels_list[i] in no_hand_pain_list:
                stim_labels[i] = 4
            elif txt_labels_list[i] in yes_back_pain_list:
                stim_labels[i] = 5
            elif txt_labels_list[i] in no_back_pain_list:
                stim_labels[i] = 6

    # extract only integer keys
    key_els = [num for num in stim_labels if isinstance(num, (int))]        
    
    return key_els

keys_frm_txt = labels_to_keys(txt_labels_list, val_list)

# print key list
print(keys_frm_txt)
len(keys_frm_txt)

## Compare txt labels to ground truth (excel file)

Import excel file

In [15]:
# import pain ratings to compare to annotations
import pandas as pd
eeg_dir = "../../../EEG DATA/"
# edf_dir = (sub_num+"EEGDATA/")

for file in os.listdir(eeg_dir):
    # print(file)
    if file.startswith(sub_num): # and os.path.isdir(file)):
        edf_dir = file
        
for file in os.listdir(eeg_dir+edf_dir):
    if file.endswith(".xlsx"):
        xfname = file

df = pd.read_excel((eeg_dir+edf_dir+'/'+xfname), sheet_name=0) # can also index sheet by name or fetch all sheets

# hand rows
# row_start_hand = 14
excel_list_hand = df['PIN PRICK PAIN SCALE '][3:].tolist() #[row_start_hand:54]

# # back rows
# row_start_back = 57
# excel_list_back = df['PIN PRICK PAIN SCALE '][row_start_back:].tolist()

# concatenate lists
excel_list = excel_list_hand# + excel_list_back

print(xfname)

PT 40 Pinprick.xlsx


#### Look for mismatch between stimulus types from excel sheet to those of the .txt file

In [16]:
ground_truth = excel_list
markers = keys_frm_txt

print('From txt file:')
print(markers)
print(len(markers))

print('GROUND TRUTH: ')
print(excel_list)
print(len(excel_list))

print('\nDo the lists match?')
mtch_ans = 'Yes.' if excel_list == markers else 'No!'
print(mtch_ans)

From txt file:
[4, 5, 4, 5, 5, 4, 5, 5, 4, 4, 8, 8, 7, 7, 8, 7, 7, 8, 7, 8, 5, 3, 5, 3, 5, 5, 3, 5, 3, 3, 6, 6, 8, 8, 6, 8, 6, 8, 6, 8, 3, 3, 4, 4, 3, 4, 4, 3, 4, 3, 7, 6, 6, 7, 6, 6, 7, 6, 7, 7]
60
GROUND TRUTH: 
[4, 5, 4, 5, 5, 4, 5, 5, 4, 4, 8, 8, 7, 7, 8, 7, 7, 8, 7, 8, 5, 3, 5, 3, 5, 5, 3, 5, 3, 3, 6, 6, 8, 8, 6, 8, 6, 8, 6, 8, 3, 3, 4, 4, 3, 4, 4, 3, 4, 3, 7, 6, 6, 7, 6, 6, 7, 6, 7, 7]
60

Do the lists match?
Yes.


In [17]:
mismatch_list = []
for i, el in enumerate(markers):
    # find mismatch
    try:
        mismatch = next( (idx, x, y) for idx, (x, y) in enumerate(zip(markers, ground_truth)) if x!=y )
    except: 
        print('No (more) mismatches found. Exiting loop.')
        break
    else:
        iss_i = mismatch[0]; print(iss_i)
        # point mismatch, extra label, or missing?
        if markers[iss_i+1:iss_i+1+3] == ground_truth[iss_i+1:iss_i+1+3]:
            markers[iss_i] = mismatch[2]
            print(f'Point error mismatch: changed label {mismatch[1]} to {mismatch[2]} in [keys_frm_txt]\n')
            
        elif markers[iss_i+1:iss_i+1+3] == ground_truth[iss_i:iss_i+3]:
            del markers[iss_i]; del txt_samps_list[iss_i]
            print(f'Extra label mismatch: expected {mismatch[2]}, deleted label {mismatch[1]} from [keys_frm_txt].\n')
            
        elif markers[iss_i+1:iss_i+1+3] == ground_truth[iss_i+1+1:iss_i+1+1+3]:
            del ground_truth[iss_i]; del txt_samps_list[iss_i]
            print(f'Missing label mismatch: expected {mismatch[2]}, deleted label {mismatch[2]} from [ground_truth] and from [txt_samps_list].\n')
            
keys_frm_txt = markers
excel_list = ground_truth

No (more) mismatches found. Exiting loop.


#### **Check resulting labels list and ground_truth**

In [29]:
if mtch_ans == 'No!':
    print('AFTER CORRECTION:\n')

    print('From txt file:')
    print(keys_frm_txt)
    print(len(keys_frm_txt))

    print('GROUND TRUTH: ')
    print(excel_list)
    print(len(excel_list))

    print('\nDo the lists match now?')
    mtch_ans = 'Yes.' if excel_list == keys_frm_txt else 'No!'
    print(mtch_ans)
else: 
    print('Labels already match.')

Labels already match.


## Create Epochs object and complete preprocessing

#### Ensure samples are in correct sampling frequency, do this in below loop

In [19]:
# create events array for input to Epochs
events_from_txt_tmp = [] # first as list 
txt_samps_list_res = txt_samps_list

for i in range(0, len(txt_samps_list)):
    txt_samps_list_res[i] = int( np.floor(txt_samps_list[i]*(res_freq/orig_freq) ) )
    events_from_txt_tmp.append([txt_samps_list_res[i], 0, keys_frm_txt[i]])

events_from_txt_arr = np.array(events_from_txt_tmp)
print(events_from_txt_arr[:5])
print(len(events_from_txt_arr))

[[313152      0      4]
 [317405      0      5]
 [320324      0      4]
 [324026      0      5]
 [326338      0      5]]
60


In [20]:
# create events to epochize data
print(f"{sub_id}\ncreating events and epochizing data...")

epochs = mne.Epochs(raw, events_from_txt_arr, tmin=-0.2, tmax=0.8, proj=True, preload=True,
                    event_repeated='drop')#, baseline=(None,0))
display.clear_output(wait=True)

epochs

# del raw # clear memory

Number of events,60
Events,3: 104: 105: 106: 107: 108: 10
Time range,-0.200 – 0.800 sec
Baseline,-0.200 – 0.000 sec


## Perform AutoReject and Save .mat Files

In [21]:
# use autoreject package to automatically clean epochs 
print(f"{sub_id}\nfinding epochs to clean...")
stim_epochs = epochs; del epochs
ar = AutoReject(random_state=42)
_, reject_log = ar.fit_transform(stim_epochs, return_log=True)
print(reject_log)
# display.clear_output(wait=True)

040.EEGDATA.1.19.2022
finding epochs to clean...
Running autoreject on ch_type=eeg


  0%|          | Creating augmented epochs : 0/64 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/64 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/60 [00:00<?,       ?it/s]

  0%|          | n_interp : 0/3 [00:00<?,       ?it/s]

...

  0%|          | Repairing epochs : 0/60 [00:00<?,       ?it/s]

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]





Estimated consensus=0.30 and n_interpolate=4


  0%|          | Repairing epochs : 0/60 [00:00<?,       ?it/s]

Dropped 8 epochs: 0, 15, 17, 24, 31, 42, 56, 58


In [22]:
# drop rejected epochs
bad_epochs_bool = reject_log.bad_epochs.tolist()
dropped_epochs_list = [i for i, val in enumerate(bad_epochs_bool) if val]
print(f'Dropped {len(dropped_epochs_list)} epochs: ', dropped_epochs_list)

data_dir = "../../../Matlab Code/Processed Data/"

# save processed epochs
print("\nSaving processed epochs...")

save_fname = sub_id[:3] + '_preprocessed_txt-epo'

stim_epochs.save(data_dir + save_fname + '.fif', 
         verbose=True, overwrite=True)

# epo_set_dir = '../../../Matlab Code/SET Data/epo/'
# mne.export.export_epochs(epo_set_dir+ save_fname + '.set', 
#                          stim_epochs, overwrite=True, verbose=True)

# save drop log
print("\nSaving drop_log as mat file...")
mdic = {"drop_log": dropped_epochs_list}
savemat(data_dir+sub_id[:3]+'_txt_drop_log.mat', mdic)

# save epo_times
print("\nSaving epo_times as mat file...")
stim_events = txt_samps_list_res
mdic = {"epo_times": stim_events}
savemat(data_dir+sub_id[:3]+'_txt_epo_times.mat', mdic)

# save stim labels
print("\nSaving stim labels as mat file...")
mdic = {"epo_times": excel_list}
savemat(data_dir+sub_id[:3]+'_txt_stim_labels.mat', mdic)

print("\nDone.")
# display.clear_output(wait=True)

Dropped 8 epochs:  [0, 15, 17, 24, 31, 42, 56, 58]

Saving processed epochs...
Overwriting existing file.

Saving drop_log as mat file...

Saving epo_times as mat file...

Saving stim labels as mat file...

Done.


In [23]:
# # adjust events_from_annot for repeated events that are dropped by MNE
# print(f"{sub_id}\nRemoving repeated epochs from annotations...")
# epochs_drop_log = list(epochs.drop_log)
# epochs_drop_ids = [idx for idx, tup in enumerate(epochs_drop_log) if epochs_drop_log[idx]]
# events_from_annot_list = events_from_annot.tolist()
# delete_multiple_elements(events_from_annot_list, epochs_drop_ids)
# events_from_annot_new = np.array(events_from_annot_list)
# print(f"\nDropped {len(raw.annotations) - len(events_from_annot_new)} repeated epochs")
# # display.clear_output(wait=True)

In [24]:
# # find epochs only from stim events
# print(f"{sub_id}\nfinding the 60 pin-prick epochs...")
# [stim_labels, StimOn_ids] = get_stim_epochs(epochs,events_from_annot_new,event_dict)

# stim_epochs = epochs[StimOn_ids]
# # display.clear_output(wait=True)

# print(stim_labels)

In [25]:
# mismatch_list = []
# for i, k in enumerate(excel_list):
#     # find mismatch
#     try:
#         mismatch = next( (idx, x, y) for idx, (x, y) in enumerate(zip(txt_labels_list, excel_list)) if x!=y )
#     except:
#         print("No more mismatches found")
#         break
#     print("index, incorrect value, correct value:", mismatch)
#     # delete mismatch
#     del key_els[mismatch[0]]
#     # save mismatch index
#     mismatch_list.append(mismatch[0])
    

In [26]:
# print(mismatch_list)

# mismatch_id=[]
# # convert mismatch to indices
# for i, m in enumerate(mismatch_list):
#     mismatch_id.append( mismatch_list[i] + i )
    
# print(mismatch_id)

In [27]:
# # drop certain stim event

# drop_list = []
# # drop_list=[12,] # C17
# # drop_list = [0,15] # PT C5
# # drop_list = mismatch_id # PT C3
# # drop_list = [6,11] # PT C2
# # drop_list = [14, 15, 16] # PT 050
# # drop_list = [40,41,66] # PT 009

# # delete from StimOn
# delete_multiple_elements(StimOn_ids, drop_list)
                         
# # delete from epochs object
# stim_epochs.drop(drop_list)

# # verify deletion
# print(len(StimOn_ids))
# stim_epochs

In [28]:
# find intersect 
# print(events_samps)

event_times_intersect = [value for value in txt_samps_list_resamp if value in events_samps]
print(event_times_intersect)
print(len(event_times_intersect))

NameError: name 'txt_samps_list_resamp' is not defined

In [ ]:
# extract indices of all events where intersection occurs to print epochs object
ab, a_ind, b_ind = np.intersect1d(events_samps,event_times_intersect, return_indices=True)

In [ ]:
print(a_ind)

In [ ]:
epochs_intersect = epochs[a_ind]

epochs_intersect